# Library

In [1]:
import platform
import psutil
import platform
import re
import subprocess

# Get System Information

In [2]:
def get_linux_distribution():
    try:
        with open("/etc/os-release", "r") as f:
            for line in f:
                if line.startswith("PRETTY_NAME="):
                    return line.split("=")[1].strip().strip('"')
    except FileNotFoundError:
        return "Distribution information not available"

def get_system_info():
    system_info = {}

    # Operating System
    os_name = platform.system()
    system_info["Operating System"] = os_name

    if os_name == "Linux":
        distribution_name = get_linux_distribution()
        system_info["Linux Distribution"] = distribution_name
    else:
        system_info["Linux Distribution"] = "N/A"

    # Processor Name
    system_info["Processor Name"] = get_processor_name()

    # RAM
    system_info["RAM (GB)"] = psutil.virtual_memory().total // (1024**3)  # in GB

    # GPU
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        system_info["GPU"] = f"{gpus[0].name} ({gpus[0].memoryTotal} MB)"
    except ImportError:
        system_info["GPU"] = "GPU information not available (GPUtil module not installed)"

    # Processor Cores
    system_info["Processor Cores"] = psutil.cpu_count(logical=False)

    # Python Version
    system_info["Python Version"] = platform.python_version()

    return system_info

def get_processor_name():
    if platform.system() == "Linux":
        with open("/proc/cpuinfo", "r") as f:
            for line in f:
                if "model name" in line:
                    return line.split(":")[1].strip()
        return "Processor information not available"
    elif platform.system() == "Windows":
        command = "wmic cpu get name"
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        output, error = process.communicate()
        if output:
            return output.decode().strip().split("\n")[1]  # Extract processor name from output
        else:
            return "Processor information not available"
    else:
        return "Processor information retrieval not supported on this platform"

In [3]:
if __name__ == "__main__":
    system_info = get_system_info()
    for key, value in system_info.items():
        print(f"{key}: {value}")

Operating System: Windows
Linux Distribution: N/A
Processor Name: AMD Ryzen 7 6800H with Radeon Graphics
RAM (GB): 31
GPU: GPU information not available (GPUtil module not installed)
Processor Cores: 8
Python Version: 3.10.13


In [4]:
!nvidia-smi

Sun Sep  1 06:28:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P0             27W /  134W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----